# 데이터 세트 가져오기

In [1]:
!mkdir data
!wget https://git.io/JtRSq -O data/download.py
!wget https://git.io/JtRSO -O data/get-all-data.sh
!chmod 755 data/get-all-data.sh
%cd data
!./get-all-data.sh
%cd ..

--2022-01-21 12:11:53--  https://git.io/JtRSq
Resolving git.io (git.io)... 18.205.36.100, 54.162.128.250, 52.204.242.176, ...
Connecting to git.io (git.io)|18.205.36.100|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_3/data/download.py [following]
--2022-01-21 12:11:53--  https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_3/data/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572 (1.5K) [text/plain]
Saving to: ‘data/download.py’

data/download.py    100%[===================>]   1.54K  --.-KB/s    in 0s      

2022-01-21 12:11:53 (21.8 MB/s) - ‘data/download.py’ saved [1572/1572]

--2022-01-21 12:11:53--  https://git.

# 데이터 세트 정제하기 (전처리)

임포트

In [2]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

매개변수 설정

In [3]:
args = Namespace(
    raw_train_dataset_csv = "/content/drive/MyDrive/공부/파이토치NLP/raw_train.csv",
    raw_test_dataset_csv = "/content/data/yelp/raw_test.csv",
    train_proportion = 0.7,
    val_proportion = 0.3, 
    output_munged_csv = "/content/data/yelp/reviews_with_splits_full.csv",
    seed = 1337
)

판다스로 원본 데이터(csv) 읽어오기

In [4]:
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])
train_reviews = train_reviews[~pd.isnull(train_reviews.review)] # 리뷰가 null인 샘플은 제외
test_reviews = pd.read_csv(args.raw_test_dataset_csv, header=None, names=['rating', 'review'])
test_reviews = test_reviews[~pd.isnull(test_reviews.review)] # 리뷰가 null인 샘플은 제외

훈련 세트와 테스트 세트의 형태 확인

In [5]:
train_reviews.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [6]:
test_reviews.head()

,rating,review
0,1,Ordered a large Mango-Pineapple smoothie. Stay...
1,2,Quite a surprise! \n\nMy wife and I loved thi...
2,1,"First I will say, this is a nice atmosphere an..."
3,2,I was overall pretty impressed by this hotel. ...
4,1,Video link at bottom review. Worst service I h...


기존 별점 1, 2점은 음성 클래스, 별점 3, 4점은 양성 클래스로 단순화 하였다.

따라서 rating class는 음성(1), 양성(2)으로 구분된다.

In [7]:
set(train_reviews.rating)

{1, 2}

훈련, 검증, 테스트 세트 만들기

In [8]:
# 별점을 기준으로 나누기
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())
# 음성 샘플들 좌르륵, 양성 샘플들 좌르륵
# 위와 같이 구분되는 딕셔너리가 만들어진다.

In [9]:
# 분할 데이터 만들기
final_list = []
np.random.seed(args.seed)

# 음성, 양성 샘플 각각에 대한 반복문
for _, item_list in sorted(by_rating.items()):

    np.random.shuffle(item_list)

    n_total = len(item_list) # 280000 (음성/양성 각각)
    n_train = int(args.train_proportion * n_total) # 0.7 * 280000 (음성/양성 각각)
    n_val = int(args.val_proportion * n_total) # 0.3 * 280000 (음성/양성 각각)

    # 샘플에 분할 속성을 추가하기
    for item in item_list[:n_train]:
        item['split'] = 'train'

    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'

    # 최종 리스트에 추가
    final_list.extend(item_list) 

In [10]:
# 테스트 세트는 그냥 분할 속성만 붙여서 최종 리스트에 추가
for _, row in test_reviews.iterrows():
    row_dict = row.to_dict()
    row_dict['split'] = 'test'
    final_list.append(row_dict)

In [11]:
# 분할 데이터를 판다스 데이터 프레임으로
final_reviews = pd.DataFrame(final_list)

In [12]:
# 분할이 어떻게 이루어졌는지 확인
final_reviews.split.value_counts()

train    392000
val      168000
test      38000
Name: split, dtype: int64

In [13]:
# 최종 리스트의 내용물 확인
final_reviews.review.head()

0    The entrance was the #1 impressive thing about...
1    I'm a Mclover, and I had no problem\nwith the ...
2    Less than good here, not terrible, but I see n...
3    I don't know if I can ever bring myself to go ...
4    Food was OK/Good but the service was terrible....
Name: review, dtype: object

In [14]:
# NULL인 리뷰가 있는지 확인
final_reviews[pd.isnull(final_reviews.review)]

,rating,review,split


리뷰 전처리

In [15]:
def preprocess_text(text):
    if type(text) == float:
        print(text)
    text = text.lower() # 소문자화
    text = re.sub(r"([.,!?])", r" \1 ", text) # 구두점 전후에 공백을 넣고
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text) # 알파벳이나 구두점이 아닌 기호를 제거

    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

In [16]:
# rating을 나타내는 1, 2를 각각 음성, 양성으로 표기
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

# 최종 리스트의 형태 확인
final_reviews.head()

,rating,review,split
0,negative,the entrance was the impressive thing about th...,train
1,negative,"i m a mclover , and i had no problem nwith the...",train
2,negative,"less than good here , not terrible , but i see...",train
3,negative,i don t know if i can ever bring myself to go ...,train
4,negative,food was ok good but the service was terrible ...,train


In [17]:
# 최종 리스트를 csv 파일로 저장
final_reviews.to_csv(args.output_munged_csv, index=False)

# 데이터 벡터화 하기

데이터 포인트 == 토큰(단어)의 집합

Vocabulary: 각 토큰을 정수에 매핑
Vectorizer: 매핑을 각 데이터 포인트에 적용하여 벡터 형태로 변환
DataLoader: 벡터로 변환한 데이터 포인트를 모델을 위해 미니배치로 모음

## Dataset 클래스

데이터셋 클래스를 만드는 전제조건
- 데이터셋이 최소한으로 정제되고 3개로 나뉠 것
- 공백을 기준으로 리뷰를 나눠서 토큰 리스트를 얻을 수 있을 것
- 샘플이 훈련, 검증, 테스트 중 어느 세트인지 표시되어 있을 것


파이토치는 이 클래스로 데이터셋을 추상화한다.
- Dataset 클래스는 추상화된 반복자이다.
- 전체적인 설계는, 데이터 포인트 하나에 벡터화 로직을 적용하는 데이터셋 클래스를 구성하는 것.

이 예제에서는 Dataset 클래스를 상속하는 ReviewDataset 클래스를 만든다.
- 이 때, __getitem__() 메소드와 __len__() 메소드를 필수적으로 구현해야 한다.
- 이 클래스 안에서, 예제 데이터셋과 함께 다양한 파이토치 유틸리티를 사용한다.
    - 그 중 하나인 DataLoader 클래스는 아래에 구현
- 이 클래스는 리뷰 텍스트를 수치 벡터로 변환하는 ReviewVectorizer 클래스에 크게 의존한다.
- 최종적으로 DataLoader 클래스가 데이터셋에서 샘플링하고 모아서 미니배치를 만든다.

In [18]:
import json
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        매개변수:
            review_df (pandas.DataFrame): 데이터셋
            vectorizer (ReviewVectorizer): ReviewVectorizer 객체
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.review_df[self.review_df.split=='val']
        self.val_size = len(self.val_df)

        self.test_df = self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.val_size),
                             'test': (self.train_df, self.train_size)}
        
        self.set_split('train')
    
    """
    staticmethod/classmethod: 정적 메소드. 클래스에서 직접 접근하며, 
    보통 객체 필드와 독립적이지만, 로직상 클래스 내에 포함되는 메소드에 사용된다.
    이 메소드 안에서는 self를 통한 접근은 불가능.
    단, 클래스메소드는 첫 인자로 클래스 자체를 넘겨줘야 한다.
    이렇게 전달받은 클래스 파라미터를 통해, 클래스 변수 등에 접근할 수 있다.
    """
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """
        데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만든다.

        매개변수:
            review_csv (str): 데이터셋의 위치
        반환값:
            ReviewDataset의 인스턴스
        """
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split=='train']

        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
        """
        데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만든다.
        캐시된 ReviewVectorizer 객체를 재사용할 때 사용한다.

        매개변수:
            review_csv (str): 데이터셋의 위치
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
        반환값:
            ReviewDataset의 인스턴스
        """
        review_df = pd.read_csv(review_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        
        return cls(review_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """
        파일에서 ReviewVectorizer 객체를 로드하는 정적 메소드

        매개변수:
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
        반환값:
            ReviewVectorizer의 인스턴스
        """
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """
        ReviewVectorizer 객체를 json 형태로 디스크에 저장한다.

        매개변수:
            vectorizer_filepath (str): ReviewVectorizer 객체의 저장 위치
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)
    
    def get_vectorizer(self):
        """
        벡터 변환 객체를 반환한다.
        """
        return self._vectorizer
    
    def set_split(self, split="train"):
        """
        데이터프레임에 있는 열을 사용해 분할 세트를 선택한다.

        매개변수:
            split (str): "train", "val", "test" 중 하나
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """
        파이토치 데이터셋의 주요 진입 메소드

        매개변수:
            index (int): 데이터 포인트의 인덱스
        반환값:
            데이터 포인트의 특성(x_data)과 라벨(y_target)로 이루어진 딕셔너리
        """
        row = self._target_df.iloc[index]

        review_vector = self._vectorizer.vectorize(row.review)
        
        rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)
        
        return {'x_data': review_vector, 'y_target': rating_index}

    def get_num_batches(self, batch_size):
        """
        배치 크기가 주어지면, 데이터셋으로 만들 수 있는 배치 개수를 반환한다.

        매개변수:
            batch_size (int)
        반환값:
            배치 개수
        """
        return len(self) // batch_size

## Vocabulary 클래스

토큰을 정수(인덱스)로 매핑하는 단계
- 일대일 매핑이 표준
    - 이는 딕셔너리 두 개를 사용하면 구현 가능

Vocabulary 클래서에서는 {인덱스:토큰} 딕셔너리와 {토큰:인덱스} 딕셔너리를 캡슐화한다.
- 사용자가 딕셔너리에 토큰을 추가하면 자동으로 인덱스를 증가시킨다.
- UNK 이라는 특별 토큰도 관리한다.
    - 테스트 시, 훈련에서 본 적 없는 토큰을 처리할 수 있도록
- 그리고 흔하지 않은 토큰도 UNK 토클으로 나타난다.
    - 이 클래스가 사용하는 메모리를 제한할 수 있도록
- add_token(): 새로운 토큰을 추가
- lookup_token(): 토큰으로 인덱스 추출
- lookup_index(): 인덱스로 토큰 추출

In [19]:
class Vocabulary(object):
    """ 매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """

    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        매개변수:
            token_to_idx (dict): 기존 {토큰:인덱스} 딕셔너리
            add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
            unk_token (str): Vocabulary에 추가할 UNK 토큰
        """
        if token_to_idx is None:
            token_to_idx = {}
        
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}

        self._add_unk = add_unk
        self._unk_token = unk_token

        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    
    def to_serializable(self):
        """ 직렬화할 수 있는 딕셔너리를 반환한다. """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    
    @classmethod
    def from_serializable(cls, contents):
        """ 직렬화된 딕셔너리에서 Vocabulary 객체를 만든다. """
        return cls(**contents)
    
    def add_token(self, token):
        """
        토큰을 기반으로 매핑 딕셔너리를 업데이트한다.

        매개변수:
            token (str): Vocabulary에 추가할 토큰
        반환값:
            index (int): 토큰에 상응하는 정수
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        
        return index
    
    def add_many(self, tokens):
        """
        토큰 리스트를 Vocabulary에 추가한다.

        매개변수:
            tokens (list): 토큰 리스트
        반환값:
            indices (list): 토큰 리스트에 상응하는 인덱스 리스트
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """
        토큰에 대응하는 인덱스를 추출한다.
        토큰이 없으면 UNK 인덱스를 반환한다.
        단, UNK 토큰을 사용하려면, 'unk_index' > 0 이어야 한다.

        매개변수:
            token (str): 찾을 토큰
        반환값:
            index (int): 토큰에 해당하는 인덱스
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    
    def lookup_index(self, index):
        """
        인덱스에 대응하는 토큰을 추출한다.

        매개변수:
            index (int): 찾을 인덱스
        반환값:
            token (str): 토큰에 해당하는 토큰
        에러:
            KeyError: 인덱스가 Vocabulary에 없을 때 발생한다.
        """
        if index not in self._idx_to_token:
            raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
        
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

## Vectorizer 클래스

입력 데이터 포인트의 토큰을 순회하면서 각 토큰을 정수로 바꾸는 단계.

그 결과는 벡터이며, 이 클래스에서 만들어진 벡터는 항상 길이가 같아야 한다.
- 여러 데이터 포인트에서 만든 벡터와 합쳐질 수 있어야 하므로
- 이를 위해 이 클래스는 Vocabulary를 캡슐화한다.

from_dataframe(): 이 클래스를 초기화하는 진입점
- 판다스 데이터프레임을 순회하면서 두 작업을 수행한다.
    - 데이터셋에 있는 모든 토큰의 빈도수 카운트
    - count > cutoff(하이퍼파라미터)인 토큰만 사용하는 Vocabulary 객체를 만든다.
        - UNK 토큰도 추가하므로, Vocabulary의 lookup_token()을 호출 할 때, 포함되지 않은 단어는 모두 unk_index를 반환한다.

vectorizer(): 이 클래스의 핵심 기능을 캡슐화한다.
- 매개변수로 리뷰 문자열을 받고, 이것의 벡터 표현을 반환한다.
    - 여기서는 '원-핫 표현'을 의미한다.
    - 즉, Vocabulary의 크기과 길이가 같고, 0과 1로 이루어진 이진 벡트를 만든다.
    - 이 벡터에서 리뷰의 단어에 해당하는 위치가 1이 된다.
    - 단, 원-핫 벡터에는 몇가지 제약이 존재한다.
        - 희소한 배열이라는 점. (한 리뷰의 고유 단어 수 < Vocabulary의 전체 단어 수)
        - 리뷰의 단어 순서를 무시한다는 점. (BoW 방식)

In [20]:
import string
from collections import Counter

class ReviewVectorizer(object):
    """ 어휘 사전을 생성하고 관리한다. """
    def __init__(self, review_vocab, rating_vocab):
        """
        매개변수:
            review_vocab (Vocabulary): 단어를 정수에 매핑하는 Vocabulary 객체
            rating_vocab (Vocabulary): 클래스 라벨을 정수에 매핑하는 Vocabulary 객체
        """
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab
    
    def vectorize(self, review):
        """
        리뷰에 대한 원-핫 벡터를 만든다.

        매개변수:
            review (str): 리뷰
        반환값:
            one_hot (np.ndarray): 원-핫 벡터
        """
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1
        
        return one_hot

    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        """
        데이터셋 데이터프레임에서 Vectorizer 객체를 만든다.

        매개변수:
            review_df (pandas.DataFrame): 리뷰 데이터셋
            cutoff (int): 빈도 기반 필터링 설정값
        반환값:
            ReviewVectorizer 객체
        """
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)

        # 점수 추가하기
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)
        
        # count > cutoff인 단어 추가하기
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
        
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)
        
        return cls(review_vocab, rating_vocab)

    @classmethod
    def from_serializable(cls, contents):
        """
        직렬화된 딕셔너리에서 ReviewVectorizer 객체를 만든다.

        매개변수:
            contents (dict): 직렬화된 딕셔너리
        반환값:
            ReviewVectorizer 객체
        """
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['rating_vocab'])

        return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)
    
    def to_serializable(self):
        """
        캐식을 위해 직렬화된 딕셔너리를 만든다.

        반환값:
            contents (dict): 직렬화된 딕셔너리
        """
        return {'review_vocab': self.review_vocab.to_serializable(),
                'rating_vocab': self.rating_vocab.to_serializable()}


## DataLoader 클래스

벡터로 변환한 데이터 포인트를 미니배치로 모으는 단계.

파이토치 내장 클래스인 DataLoader는 이 작업을 편하게 해준다.
- 매개변수로 Dataset(이 예제에서는 ReviewDataset), batch_size 등 몇 가지 편리한 키워드를 받는다.
- 이 클래스의 객체는, Dataset 클래스가 제공한 데이터 포인트를 순회하는 반복자이다.
    - Dataset 클래스를 상속할 때, __getitem__()과 __len__()를 구현한 이유
    - 이를 통해 DataLoader 클래스가 인덱스를 사용해 데이터셋을 순회한다.

generate_batches():
- DataLoader를 감싸는 함수.
- CPU와 GPU간의 데이터 전환을 간편하게 하는 생성자(generator)이다.

In [21]:
from torch.utils.data import DataLoader

def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'):
    """
    파이토치 DataLoader 객체를 감싸는 제너레이터 함수.
    각 텐서를 지정된 장치로 이동시킨다.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

# 퍼셉트론 분류기 만들기

code03에서 구현한 퍼셉트론을 다시 구현한 모델을 사용하자. 모델(클래스)명은 ReviewClassifier.

ReviewClassifier은 파이토치의 Module 클래스를 상속하고 단일 출력을 만드는 Linear 층을 하나 생성한다.
- 이진 분류 문제(음/양)이므로, 출력이 하나이다.
- 마지막 비선형 활성화 함수는 시그모이드이다.

forward():
- 선택적으로 시그모이드를 적용하는 매개변수가 있다.
    - 이진 분류 문제에서 가장 적절한 손실 함수는 이진 크로스 엔트로피 손실(torch.nn.BCELoss())
        - 이 함수는 이진 확률을 계산한다.
        - 하지만 시그모이드와 손실 함수를 함께 사용할 때는 수치 안정성 이슈가 있다.
    - 파이토치는 시그모이드와 BCE 손실 함수를 합친, 수치적으로 안정된 계산을 위한 BCEWithLogitsLoss() 함수를 제공한다.
        - 이 함수를 사용하려면 출력에 시그모이드를 적용하면 안된다.
        - 따라서 forward()의 기본값은 시그모이드를 적용하지 않는다.
        - 다만, 사용자가 확률을 확인하려면 시그모이드를 씌워야 하기 때문에, 옵션으로 남겨두었다.

In [22]:
import torch
import torch.nn as nn

class ReivewClassifier(nn.Module):
    """ 간단한 퍼셉트론 기반 분류기"""

    def __init__(self, num_features):
        """
        매개변수:
            num_features (int): 입력 특성 벡터의 크기
        """ 
        super(ReivewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, 
                             out_features=1) # 출력은 하나
        
    def forward(self, x_in, apply_sigmoid=False):
        """ 
        분류기의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape: (batch, num_features)
            apply_sigmoid (bool): 시그모이드 활성화 함수를 위한 플래그
                BCEWithLogitsLoss() 함수를 이용하려면 False로 지정
        
        반환값:
            결과 텐서 tensor.shape: (batch, )
        """
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)

        return y_out

# 모델 훈련 전 환경 조성하기

## 헬퍼 함수 1 정의하기

In [23]:
def set_seed_everywhere(seed, cuda):
    """ 항상 동일한 결과를 위해, 랜덤 시드를 통일 """
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

import os
def handle_dirs(dirpath):
    """ 디렉토리가 존재하지 않으면 추가하기 """
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

## 모델 생성과 훈련 과정에서 고수준의 결정을 조정하기 편하도록 args 객체를 만들기

In [31]:
args = Namespace(
    # 경로 설정
    frequency_cutoff = 25, # Vectorizer 클래스에서 쓸 cutoff
    review_csv = '/content/data/yelp/reviews_with_splits_full.csv',
    model_state_file = 'model.pth', # 모델 상태를 저장할 파일 이름
    vectorizer_file = 'vectorizer.json', # vectorizer을 저장할 파일 이름
    save_dir = 'model_storage/ch3/yelp', # vectorizer_file과 model_state_file을 저장할 디렉토리

    # 모델 하이퍼파라미터 없음

    # 훈련 하이퍼파라미터
    batch_size = 128,
    early_stopping_criteria = 5,
    learning_rate = 0.01, # 원래 0.001인데 에포크 줄여서 올림
    num_epochs = 5, # 원래 100인데 너무 오래 걸려서 줄임
    seed = 1337,

    # 실행 옵션
    catch_keyboard_interrupt = True,      # 키보드 인터럽트 처리
    cuda = True,                          # GPU 이용
    expand_filepaths_to_save_dir = True,  # filepaths 앞에 save_dir을 부가
    reload_from_files = False,            # 체크포인트에서 훈련을 다시 시작할지 말지
)

## GPU 설정 및 시드, 파일 경로 설정

In [32]:
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

    print("파일 경로: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

print("CUDA 사용여부: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")

# 재현성을 위한 시드 설정
set_seed_everywhere(args.seed, args.cuda)

# 디렉토리 처리
handle_dirs(args.save_dir)

파일 경로: 
	model_storage/ch3/yelp/vectorizer.json
	model_storage/ch3/yelp/model.pth
CUDA 사용여부: True


## 헬퍼 함수 2 정의하기

In [33]:
def make_train_state(args):
    """ 훈련 상태를 초기화하는 함수 """
    return {
        'stop_early': False,
        'early_stopping_step': 0,
        'early_stopping_best_val': 1e8,
        'learning_rate': args.learning_rate,
        'epoch_index': 0,
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'test_loss': -1,
        'test_acc': -1,
        'model_filename': args.model_state_file
    }

def update_train_state(args, model, train_state):
    """
    훈련 상태를 업데이트한다.
    성능이 향상되면, 현재 모델을 저장하여 최상의 모델을 사용할 수 있도록 한다.

    매개변수:
        args: 메인 매개변수
        model: 훈련할 모델
        train_state: 훈련 상태를 담은 딕셔너리
    반환값:
        train_state: 새로운 훈련 상태를 담은 딕셔너리
    """
    # 적어도 한 번 모델을 저장한다.
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # 성능이 향상되면 모델을 저장한다.
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # 손실이 증가하면
        if loss_t >= train_state['early_stopping_best_val']:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
        # 손실이 감소하면
        else:
            # 최상의 모델 저장
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
            
            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0
        
        # 조기 종료 여부 확인
        train_state['stop_early'] = train_state['early_stopping_step'] >= args.early_stopping_criteria
    
    return train_state

def compute_accuracy(y_pred, y_target):
    """ 예측값과 타깃을 비교하여 정확도 계산 """
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()

    return n_correct / len(y_pred_indices) * 100

## 데이터셋과 Vectorizer 준비하기

In [34]:
import torch.optim as optim

# 데이터셋과 Vectorizer 만들기
if args.reload_from_files:
    # 이전에 만든 Vectorizer 로드
    print("데이터셋과 Vectorizer를 로드합니다.")
    dataset = ReviewDataset.load_dataset_and_load_vectorizer(args.review_csv, args.vectorizer_file)
else:
    print("데이터셋을 로드하고 Vectorizer를 만듭니다.")
    # 새로운 Vectorizer 로드
    dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# 모델 만들기
classifier = ReivewClassifier(num_features = len(vectorizer.review_vocab))
classifier = classifier.to(args.device) # 모델과 데이터가 같은 장치에 있어야 한다.

# 손실함수(BCE)와 옵티마이저(Adam)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)
train_state = make_train_state(args)

데이터셋을 로드하고 Vectorizer를 만듭니다.


# 모델 훈련시키기

훈련 과정의 핵심
- 모델을 만들기
- 데이터셋을 순회하기
- 입력 데이터에서 모델의 출력을 계산하기
- 손실을 계산하기
- 손실에 비례하여 모델을 수정하기

## 훈련 시작 및 반복

앞서 초기화한 객체를 사용해 모델의 성능이 높아지도록 모델 파라미터를 업데이트한다.

구체적인 훈련 반복은 두 개의 반복문으로 구성된다.
1. 외부 반복문은 내부 반복문을 (args.num_epoch)만큼 반복한다.
2. 내부 반복문은 데이터셋의 미니배치에 대해서 반복을 수행한다.
    - 훈련 세트를 순회하는 반복문과, 검증 세트를 순회하는 반복문으로 나뉜다.
    - 미니배치마다 손실을 계산하고, 옵티마이저는 모델 파라미터를 업데이트한다.

자세한 주석은 코드상에 적기로

In [35]:
# tqdm(진행률 프로세스바 제공 모듈)을 이용하여
# 전체적인 학습 과정과 훈련 데이터 학습 과정 및 Validation 데이터셋을 통한 검증 과정을
# 프로세스바로 시각화할 수 있도록 하기
import tqdm

epoch_bar = tqdm.notebook.tqdm(desc='training routine',
                               total=args.num_epochs, position=0)
dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                               total=dataset.get_num_batches(args.batch_size),
                               position=1, leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                             total=dataset.get_num_batches(args.batch_size),
                             position=1, leave=True)

try:
    for epoch_index in range(args.num_epochs): # 외부 반복문
        # 훈련 상태의 에포크 설정
        train_state['epoch_index'] = epoch_index 

        """"""""""""""" 훈련 세트 순회 """""""""""""""
        # 훈련 세트와 배치 제너레이터 준비
        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                           batch_size = args.batch_size,
                                           device = args.device)
        # 배치 간의 손실과 정확도를 기록할 변수를 0으로 초기화
        running_loss = 0.0
        running_acc = 0.0
        # 분류기(모델)가 훈련 모드에 있음을 명시
        # 모델 파라미터를 수정할 수 있고, 드롭아웃 등 규제를 적용할 수 있음
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 내부 반복문: 훈련 과정은 5단계로 이루어진다.
            
            # ------------------------------------------------
            # 1단계: 옵티마이저의 그래디언트를 0으로 초기화
            optimizer.zero_grad()

            # 2단계: 모델의 출력을 계산
            y_pred = classifier(x_in=batch_dict['x_data'].float())

            # 3단계: 손실을 계산
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_batch = loss.item()
            running_loss += (loss_batch - running_loss) / (batch_index + 1) # 이동평균손실 업데이트

            # 4단계: 손실을 사용하여 그래디언트 계산, 각 파라미터에 그래디언트 전파
            loss.backward()

            # 5단계: 옵티마이저로 가중치 업데이트
            optimizer.step()

            # 추가 단계: 정확도를 계산
            acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_batch - running_acc) / (batch_index + 1) # 이동평균정확도 업데이트
            # ------------------------------------------------

            # 진행 바 업데이트
            train_bar.set_postfix(loss = running_loss,
                                  acc = running_acc,
                                  epoch = epoch_index)
            train_bar.update()

        # 훈련 상태 업데이트
        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        """"""""""""""" 검증 세트 순회 """""""""""""""
        # 검증 세트와 배치 제너레이터 준비
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                           batch_size = args.batch_size,
                                           device = args.device)
        # 배치 간의 손실과 정확도를 기록할 변수를 0으로 초기화
        running_loss = 0.0
        running_acc = 0.0
        # 분류기(모델)가 훈련 모드에 있지 않음을 명시
        # 모델 파라미터를 수정할 수 없고, 드롭아웃 등 규제를 적용할 수 없음
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 내부 반복문: 검증 과정에서는 역전파 X --> 역전파 관련 단계 X
            
            # ------------------------------------------------
            # 1단계: 모델의 출력을 계산
            y_pred = classifier(x_in=batch_dict['x_data'].float())

            # 2단계: 손실을 계산
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_batch = loss.item()
            running_loss += (loss_batch - running_loss) / (batch_index + 1) # 이동평균손실 업데이트

            # 정확도를 계산
            acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_batch - running_acc) / (batch_index + 1) # 이동평균정확도 업데이트
            # ------------------------------------------------

            # 진행 바 업데이트
            val_bar.set_postfix(loss = running_loss,
                                acc = running_acc,
                                epoch = epoch_index)
            val_bar.update()

        # 훈련 상태 업데이트
        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier, train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        # 조기종료 조건에 도달하면 브레이크
        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")

training routine:   0%|          | 0/5 [00:00<?, ?it/s]

split=train:   0%|          | 0/3062 [00:00<?, ?it/s]

split=val:   0%|          | 0/1312 [00:00<?, ?it/s]

# 테스트 세트 평가

학습 과정에서 가장 성능이 좋은 모델은 args.model_state_file에 저장되어 있다.

다음에 모델 불러올 때에는 구글드라이브 링크 "/content/drive/MyDrive/공부/파이토치NLP/model.pth"를, "/model_storage/ch3/yelp" 로 복사해주면 된다.

In [38]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 게산한다.
classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size = args.batch_size,
                                   device = args.device)
running_loss = 0.0
running_acc = 0.0
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 1단계: 모델의 출력을 계산
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    # 2단계: 손실을 계산
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [39]:
print("테스트 손실: {:.3f}".format(train_state['test_loss']))
print("테스트 정확도: {:.2f}".format(train_state['test_acc']))

테스트 손실: 0.135
테스트 정확도: 95.12


# 추론 및 해석

추론: 테스트셋을 통한 평가와는 달리, 새로운 데이터를 이용하여 잘 동작하는지 정성적으로 평가하는 것

In [44]:
def predict_rating(review, classifier, vectorizer, decision_thresold=0.5):
    """
    리뷰로 점수(양/음) 예측하기

    매개변수:
        review (str): 리뷰 텍스트
        classifier (ReviewClassifier): 훈련된 모델
        vectorizer (ReviewVectorizer): Vectorizer 객체
        decision_thresold (float): 클래스를 나눌 결정 경계
    """
    review = preprocess_text(review)

    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))

    probability_value = torch.sigmoid(result).item()
    index = 1 # 긍정 인덱스 1
    if probability_value < decision_thresold:
        index = 0 # 부정 인덱스 0
    
    return vectorizer.rating_vocab.lookup_index(index)

In [45]:
test_review = "This is a pretty awesome book!"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_thresold=0.5)

print("{} --> {}".format(test_review, prediction))

This is a pretty awesome book! --> positive


해석: 가중치를 분석하여 올바른 값인지 평가하는 것. 훈련이 끝난 후 최종적으로 모델이 잘 동작하는지 알아보는 과정
- 퍼셉트론의 가중치는 어휘 사전의 한 단어와 정확하게 대응하므로 이를 쉽게 확인할 수 있다.

In [46]:
# 가중치의 모양을 확인하기 (단어 수?)
classifier.fc1.weight.shape

torch.Size([1, 24712])

In [47]:
# 가중치 정렬
fc1_weights = classifier.fc1.weight.detach()[0]
_, indices = torch.sort(fc1_weights, dim=0, descending=True)
indices = indices.numpy().tolist()

# 긍정 상위 20개 단어
print("긍정 리뷰에 영향을 미치는 단어: ")
print("--------------------------------")
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

print("====\n\n\n")

# 부정 상위 20개 단어
print("부정 리뷰에 영향을 미치는 단어: ")
print("--------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

긍정 리뷰에 영향을 미치는 단어: 
--------------------------------
yummo
couverte
yummm
fcrlich
excellently
mmmmmmmm
crunchier
ndelicious
exceeded
aubergine
superbe
immaculately
tubing
haters
quibble
nshow
maz
licious
montagu
pavilions
====



부정 리뷰에 영향을 미치는 단어: 
--------------------------------
ception
unusable
nuh
discriminated
npass
demerits
blandest
dishonesty
wrinkled
nnope
discusting
nmeh
underwhelmed
conclusions
clump
poisoning
acetone
methinks
rigged
mediocore
